In [4]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train_validation, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.80)

In [6]:
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
model = SLIM_BPR_Cython

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])
earlystopping_kwargs =  {"validation_every_n": 5,
                              "stop_on_validation": True,
                              "evaluator_object": evaluator_validation,
                              "lower_validations_allowed": 5,
                              "validation_metric": "MAP",
                              }

EvaluatorHoldout: Ignoring 2635 (20.8%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [7]:
import optuna as op
def objective(trial):
    # Optuna suggest params
    topK = trial.suggest_int("topK", 5, 1000)
    symmetric = trial.suggest_categorical("symmetric", [True, False])
    sgd_mode = trial.suggest_categorical("sgd_mode", ["sgd", "adagrad", "adam"])
    epochs = trial.suggest_categorical("epochs", [1500])
    lambda_i = trial.suggest_loguniform("lambda_i", 1e-5, 1e-2)
    lambda_j = trial.suggest_loguniform("lambda_j", 1e-5, 1e-2)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-4, 1e-1)
    
    recommender = model(URM_train)
    recommender.fit(**trial.params, **earlystopping_kwargs)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [9]:
best_params = {
    "topK": 100,
    "symmetric": True,
    "sgd_mode": "sgd",
    "epochs": 1500,
    "lambda_i": 0.0025,
    "lambda_j": 0.00025,
    "learning_rate": 0.05,
}
# "topK": Integer(5, 1000), "epochs": Categorical([1500]), "symmetric": Categorical([True, False]),  "sgd_mode": Categorical(["sgd", "adagrad", "adam"]), "lambda_i": Real(low = 1e-5, high = 1e-2, prior = 'log-uniform'), "lambda_j": Real(low = 1e-5, high = 1e-2, prior = 'log-uniform'), "learning_rate": Real(low = 1e-4, high = 1e-1, prior = 'log-uniform')
study_name = "SLIMBPR-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
study.enqueue_trial(best_params)
study.optimize(objective, n_trials=30)

[I 2023-11-23 16:56:14,064] Using an existing study with name 'SLIMBPR-study' instead of creating a new one.
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_9092/2976677453.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lambda_i = trial.suggest_loguniform("lambda_i", 1e-5, 1e-2)
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_9092/2976677453.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lambda_j = trial.suggest_loguniform("lambda_j", 1e-5, 1e-2)
/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_9092/2976677453.py:10: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See http

SLIM_BPR_Recommender: URM Detected 453 ( 3.6%) users with no interactions.
SLIM_BPR_Recommender: URM Detected 332 ( 1.5%) items with no interactions.


ModuleNotFoundError: No module named 'Recommenders.SLIM.Cython.SLIM_BPR_Cython_Epoch'

In [9]:
study.best_params
# epoch = 260

{'sgd_mode': 'adagrad',
 'num_factors': 219,
 'batch_size': 32,
 'positive_reg': 0.016211221706684466,
 'negative_reg': 0.00012672106970228173,
 'learning_rate': 0.08036400470449359}

In [10]:
final = model(URM_train_validation)
final.fit(**study.best_params, epochs=260, random_seed=1234)

MatrixFactorization_BPR_Cython_Recommender: URM Detected 237 ( 1.9%) users with no interactions.
MatrixFactorization_BPR_Cython_Recommender: URM Detected 111 ( 0.5%) items with no interactions.
MF_BPR: Processed 12640 (100.0%) in 0.69 sec. MSE loss 5.71E-01. Sample per second: 18202
MF_BPR: Epoch 1 of 260. Elapsed time 0.11 sec
MF_BPR: Processed 12640 (100.0%) in 0.81 sec. MSE loss 1.47E+00. Sample per second: 15666
MF_BPR: Epoch 2 of 260. Elapsed time 0.22 sec
MF_BPR: Processed 12640 (100.0%) in 0.92 sec. MSE loss 2.17E+00. Sample per second: 13711
MF_BPR: Epoch 3 of 260. Elapsed time 0.34 sec
MF_BPR: Processed 12640 (100.0%) in 1.04 sec. MSE loss 2.66E+00. Sample per second: 12170
MF_BPR: Epoch 4 of 260. Elapsed time 0.46 sec
MF_BPR: Processed 12640 (100.0%) in 0.15 sec. MSE loss 3.14E+00. Sample per second: 83931
MF_BPR: Epoch 5 of 260. Elapsed time 0.57 sec
MF_BPR: Processed 12640 (100.0%) in 0.27 sec. MSE loss 3.65E+00. Sample per second: 47410
MF_BPR: Epoch 6 of 260. Elapsed time

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users=[])
evaluator_test.evaluateRecommender(final)

EvaluatorHoldout: Ignoring 2218 (17.6%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users
EvaluatorHoldout: Processed 10420 (100.0%) in 5.95 sec. Users per second: 1750


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.031555                 0.061472  0.054007  0.012733    0.025207   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.098538  0.050914  0.039835  0.244626      0.111836  ...   
 
        COVERAGE_USER COVERAGE_USER_HIT USERS_IN_GT DIVERSITY_GINI  \
 cutoff                                                              
 10          0.824498          0.201693    0.824498       0.054326   
 
        SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL RATIO_DIVERSITY_GINI  \
 cutoff                                                                   
 10             9.14083                   0.992109             0.155907   
 
        RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY RATIO_NOVELTY  
 cutoff                                   

In [ ]:
## MAP = .0127328